## 국가보안기술연구소(NSRI)
- 함수의 시작 구분 프로젝트 by ISYS Lab

## Input 
- objdump -d 로 바이트를 뽑아서 저장한 .txt 파일 (16진수 hex 코드 & 함수이름 섞여있음)

## output
- 해당 바이트코드와 함수의 시작여부(bin) 형태의 csv 파일

In [1]:
# preprocessing 함수 (0, 1) 이중분류
# 16진수 가져와서 함수가 적혀있는 행은 버리고 , 그 다음항목 1로 마킹
# 500000만번 마다 체크
def preprocessing(k):
    p = pd.DataFrame(np.array(k.split()))
    p['label'] = 0 
    p.columns = ['bin', 'label']
    
    f_index = [] # 함수 위치 index 담은 list 
    f_name = [] # 함수 이름 담은 list
    for idx in range(0, len(p)):
        # 진행현황 체크
        if idx % 500000 == 0:
            print(idx,'Finish', end=' ') # 진행속도 체크 함수

        if len(str(p['bin'][idx])) !=2: # 칸의 이름이 2글자가 아니면
            f_index.append(idx); f_name.append(str(p['bin'][idx])) # 함수 이름 저장 list
            p['label'][idx+1] = 1 # bin 항목 1로 마킹
            #print(idx, end=' ')#, p['num'][idx])
            
    # 함수이름 행들 버리기
    drop_index = p['bin'][f_index].index # drop 할 index
    p = p.drop(drop_index, 0) # drop 행 버리기
    print('전체 행 갯수:', len(p), '함수갯수:', len(drop_index),'byte갯수:', len(p)-len(drop_index))
    
    return f_index, f_name, p # 함수 위치 index list , 함수 이름 list, 최종 df 

In [6]:
######################### main 함수
# gcc v_start ~ v_end - optimization 0~4
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

#v_start, v_end = 6, 7 # version 정보 - ex 6,10 = gcc 6버전부터 ~ 9버전까지
v_start, v_end = 6, 7
o_start, o_end = 0, 4 # optimizaton 정보 - ex 0, 4 = 최적화 O0, O1, O2, O3 까지

dir_path = 'data/binutils_stripfile_txt/'

f_index_ls = [] # 함수 index ls
f_name_ls = [] # 함수 이름 ls
df_ls = [] # 데이터프레임 ls

# gcc version 별로
for version in range(v_start, v_end):
    for optimization in range(o_start, o_end):    
        # 파일 경로 , gcc 버전, op 버전 지정
        start_time = time.time() # 소요시간 측정용
        file_name = dir_path +'binutils-2.34-gcc' + str(version) + '-O' + str(optimization) + '-x86_strip.txt'
        
        # 파일 open
        print(file_name, '시작')
        f = open(file_name, 'r')
        
        # 파일 읽어들이기 완료, 전처리 시작
        index_ls, name_ls, p_ls = preprocessing(f.read())
        f_index_ls.append(index_ls); f_name_ls.append(name_ls); df_ls.append(p_ls)
        
        # 파일 전처리 종료
        end_time = time.time() # 소요시간 측정용
        print(file_name, '완료')
        print('걸린시간:', np.round(end_time-start_time,1),'초\n\n')

data/binutils_stripfile_txt/binutils-2.34-gcc6-O0-x86_strip.txt 시작
0 Finish 500000 Finish 1000000 Finish 1500000 Finish 2000000 Finish 2500000 Finish 3000000 Finish 3500000 Finish 4000000 Finish 4500000 Finish 5000000 Finish 5500000 Finish 6000000 Finish 6500000 Finish 7000000 Finish 7500000 Finish 8000000 Finish 8500000 Finish 9000000 Finish 9500000 Finish 10000000 Finish 10500000 Finish 11000000 Finish 11500000 Finish 12000000 Finish 12500000 Finish 13000000 Finish 13500000 Finish 전체 행 갯수: 13936744 함수갯수: 2146 byte갯수: 13934598
data/binutils_stripfile_txt/binutils-2.34-gcc6-O0-x86_strip.txt 완료
걸린시간: 369.2 초


data/binutils_stripfile_txt/binutils-2.34-gcc6-O1-x86_strip.txt 시작
0 Finish 500000 Finish 1000000 Finish 1500000 Finish 2000000 Finish 2500000 Finish 3000000 Finish 3500000 Finish 4000000 Finish 4500000 Finish 5000000 Finish 5500000 Finish 6000000 Finish 6500000 Finish 7000000 Finish 7500000 Finish 8000000 Finish 8500000 Finish 9000000 Finish 9500000 Finish 10000000 Finish 1050000

## 파일저장 

In [7]:
####################### 각 gcc-version-optimization 별 한번에 저장 

idx = o_start
for d in df_ls:    
    # 한번에 저장
    d.to_csv('gcc'+str(v_start)+'_'+str(idx) +'_32_strip.csv')
    print('gcc'+str(v_start)+'_'+str(idx)+'_32_strip.csv', '생성완료')
    idx+=1

gcc6_0_32_strip.csv 생성완료
gcc6_1_32_strip.csv 생성완료


In [9]:
# test read
gcc6_0_32_strip = pd.read_csv('data/gcc6_0_32_strip.csv')
gcc6_1_32_strip = pd.read_csv('data/gcc6_1_32_strip.csv')
gcc6_2_32_strip = pd.read_csv('data/gcc6_2_32_strip.csv')
gcc6_3_32_strip = pd.read_csv('data/gcc6_3_32_strip.csv')

print(gcc6_0_32_strip.shape, gcc6_1_32_strip.shape)
print(gcc6_2_32_strip.shape, gcc6_3_32_strip.shape)
gcc6_2_32_strip.head()


(13936744, 3) (10532416, 3)
(11076816, 3) (13485088, 3)


,Unnamed: 0,bin,label
0,1,53,1
1,2,83,0
2,3,ec,0
3,4,08,0
4,5,e8,0


In [12]:
print(gcc6_0_32_strip['label'].value_counts())
print(gcc6_1_32_strip['label'].value_counts())
print(gcc6_2_32_strip['label'].value_counts())
print(gcc6_3_32_strip['label'].value_counts())


0    13934630
1        2114
Name: label, dtype: int64
0    10530377
1        2039
Name: label, dtype: int64
0    11074725
1        2091
Name: label, dtype: int64
0    13482985
1        2103
Name: label, dtype: int64
